# Básico de Julia

Para aquecimento, segue uma breve quanto a variáveis em Julia

Julia não é muito restritiva com relação aos nomes de variáveis e faz uso do padrão unicode. Na linha de comando a variável $\delta$, por exemplo, pode ser inserida usando o padrão latex \delta seguido de TAB - e outros detalhes (como expoentes) podem adicionados de maneira análoga. Se você tem um arquivo e não sabe como inserir um símbolo contido no arquivo, o comando `?símbolo` te diz (onde "símbolo" é o símbolo que quer escrever)

In [12]:
δ̂  = 1
δ̂ += 2
αⁱ = δ̂ 
x,y,z = pi, [1:20;], "uma string"  # sem o ;, [1:20] não é "aberto" 

(π, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], "uma string")

#### Dá para fazer "encadeamento":

In [13]:
x = y = 5
0 < x < 10

true

Operadores (como o `+`) também podem ser usados como variáveis:
`(+) = f` altera a função de adição. Mais detalhes sobre funções virão depois.

### Convenções estilísticas
1. variáveis, funções e macros em caixa baixa;
3. funções que modificam seus argumentos devem terminar com um `!`;
2. nomes de Tipos e Módulos capitalizados;

### Maiores Diferenças
Na maior parte, a sintaxe de Julia é semelhante a de outras linguagens de alto-nível atuais, com algumas diferenças que serão notadas em um programa-exemplo posteriormente.

Antes disso, é oportuno notar algumas das principais diferenças, que tornam Julia especialmente diferente das demais, o seu sistema de tipos e o seu sistema de metaprogramação.

#### Tipos
_"In Julia, types are themselves run-time objects, and can also be used to convey information to the compiler."_

Julia é dinâmicamente tipada e, por padrão, uma variável criada sem anotação de tipo (como visto nos exemplos acima) é do tipo `Any`. O tipo `Any` está no apex do grafo de tipos com que a linguagem trabalha (ie. todo tipo é subtipo de `Any`) e uma variável desse tipo pode assumir qualquer valor. Muitas funções úteis podem ser escritas assim, mas é considerado boa prática gradualmente adicionar anotações de tipo ao código. Isso auxilia tanto a eficiência do programa, o _debugging_ e a sua compreensão por humanos.

Esse tipo de anotação é feita com `::Tipo` (veremos exemplos a seguir).

In [54]:
x::Number

5.6

Tipos compostos (chamados _structs_, _records_ ou _objetos_ em outras linguagens)são coleções de campos nomeados, que podem ser tratados como um valor único. 

Em Julia, tipos compostos não comportam métodos, apenas valores (diferente de linguagens orientadas a objeto padrão). Isso é necessário para que o _multiple dispatch_ funcione bem (mais disso será visto a seguir).

Veja exemplo de definição de um tipo composto:

In [55]:
struct Foo
    bar            # <- de tipo Any, pode receber qualquer valor
    baz::Int       # <- precisa receber valor Int
    qux::Float64
end

Foo((), 4.5, 5)    # <- 4.5 não é Int

InexactError: InexactError: Int64(4.5)

O código acima dá erro porque 4.5 não é Int. Note que a definição de um tipo composto `Foo` vem com a definição de seu construtor homônimo e que os campos  de Foo são acessíveis com a sintaxe `Foo.campo`:

In [28]:
foo = Foo([1:5;], 8, 4)
foo.bar

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

Objetos compostos definidos com `struct` são imutáveis:

In [24]:
foo.bar = pi

ErrorException: setfield! immutable struct of type Foo cannot be changed

Um objeto imutável pode conter campos mutáveis em seus campos. Esses continuam mutáveis:

In [35]:
foo.bar[1] = 9  # note, o index começa em 1
foo.bar

5-element Array{Int64,1}:
 9
 2
 9
 4
 5

Tipos compostos (structs) podem ser declarados mutáveis com `mutable` (usando `mutable struct`), mas isso geralmente é desaconselhado (e gera código menos eficiente).

Tipos são, conceitualmente, conjuntos. Assim, podemos fazer a união:

In [46]:
struct Grid1
    nx::Int
    ny::Int
    M
end

struct Description
    height::Int
    width::Int
end

Model = Union{Grid1, Description}

Union{Description, Grid1}

Julia também permite a definição de **tipos paramétricos**. No código a seguir, `T` pode ser qualquer tipo válido.

A última linha mostra que `Grid2{Array{Float64,2}}` é um subtipo de `Grid2`

In [49]:
struct Grid2{T}
    nx::Int
    ny::Int
    M::T
end

Grid2{Array{Float64,2}} <: Grid2     # A <: B lê-se "A, subtipo de B"

true

Mais será dito sobre métodos depois, mas, como um primeiro exemplo, note que o seguinte método recebe como argumentos qualquer G que seja subtipo de `Grid2` com matriz bidimensional:

In [52]:
function norm(g::Grid2{Array{<:Real, 2}})  # <- note o 'Real', supertipo de float64, float16 etc e Int e etc
   # calcula norma 
end

norm (generic function with 1 method)

### Funções, Métodos e _Multiple Dispatch_ 

Funções em Julia são objetos que mapeiam tuplas $(a_1, ..., a_n)$ a algum valor. Elas não são como funções no sentido matemático, já que podem alterar o valor do argumento passado. Seguem três exemplos de definições de funções para realizar a mesma operação. O último deles usa a sintaxe de uma função anônima `(a_1, ..., a_n) -> (operações com a_i)`

In [68]:
function f(a,b)
    a + b
end

g(a,b) = a + b

h = (a,b) -> a + b

Σ(a,b) = a + b       # funções também fazem uso de Unicode

#9 (generic function with 1 method)

Sem os parenteses, funções são objetos como quaisquer outros, podendo ser passados para cá e para lá, assim como operadores (como +, -, *) - que também são funções.

Isto é, funções são _membros de primeira classe_, assim como tipos.

Julia tem várias características que tornam a manipulação de funções agradável. 

A seguir, são apresentados alguns exemplos - para maiores explicações, consulte [a seção sobre funções do manual](https://docs.julialang.org/en/v1/manual/functions/)

##### Funções com *keyword-arguments*
Ao chamar essas funções, pode-se especificar apenas um subconjunto qualquer
dos argumentos, pelo nome. Note que **Esses argumentos não participam do multiple dispatch** (que veremos a seguir):

In [ ]:
function plot(x, y; style="solid", width=1, color="black")
    ###
end
# o ; é opcional ao chamar (mas é comum)
plot(x, y, width=2) 

##### Do syntax:
Esse *do* cria uma função anônima (que vou chamar de λ)e a passa como primeiro argumento da função que precede o *do*.
O código a seguir equivale a map(λ, [A,B,C]), onde λ é em função de x lembre-se que 
 >map(λ, [l1, ..., ln]) := [λ(l1), ..., λ(ln)]

In [ ]:
map([A, B, C]) do x
    if x < 0 && iseven(x)
        return 0
    elseif x == 0
        return 1
    else
        return x
    end
end

#### Composição e encadeamento de funções

In [ ]:
# Composição de funções:
# (f ∘ g)(x) := f(g(x))
(sqrt ∘ +)(3,6) # = sqrt(3 + 6)


# Encadeamento de funções (piping)
# o resultado da operação à esquerda é passado como argumento à função à direita
1:10 |> sum |> sqrt # o mesmo que (sqrt ∘ sum)(1:10)

#### Vetorização de funções
Só adicione um "."

In [66]:
A = [1.0, 2.0, 3.0]
sin.(A)

3-element Array{Float64,1}:
 0.8414709848078965
 0.9092974268256817
 0.1411200080598672

##### Funções com número de argumentos indeterminado (funções vararg)

In [64]:
bar(a,b,x...) = (a,b,x)
bar(1,2)        # -> (1, 2, ())
bar(1,2,3)      # -> (1, 2, (3,))
bar(1, 2, 3, 4) # -> (1, 2, (3, 4))

(1, 2, (3, 4))

>_"Although it seems a simple concept, multiple dispatch on the types of values is perhaps the single most powerful and central feature of the Julia language. Core operations typically have dozens of methods (...)"_

Uma mesma _função_ pode ser implementada de várias formas diferentes. Por exemplo, as operações computacionais para somar dois inteiros são bem diferentes das para somar dois _float_, mas toda soma é realizada com o +.

Em Julia, uma função não precisa ser definida inteiramente de uma vez - alguém pode querer primeiro implementar a soma para inteiros, e depois para _float, digamos. Isso é feito definindo métodos para a função, que são distinguidos segundo sua assinatura de tipo. 

A escolha de qual método usar é chamado de _dispatch_. Em Julia é utilizado _multiple dispatch_, o que significa que a escolha do método é feita com base nos tipos de todos os argumentos - e não só no tipo do primeiro, como é típico em linguagens orientadas a objeto.

O _multiple dispatch_ funciona resumidamente da seguinte forma:
1. Ao definir uma função como fizemos acima, definimos uma função com 1 método;
2. Para adicionar outro método à função, utiliza-se a mesma sintaxe, mas utilizando uma assinatura de tipos diferente;
3. Ao chamar a função, é escolhido o método com a assinatura de tipos mais específico que pareia com o da tupla de argumentos - note que, se esse método não é único, ocorre um erro.

Vejamos exemplos:

In [86]:
# note que 2x = 2*x

g(x::Float64, y::Float64) = 2x + y    # se aplica somente quando x e y são Float64
# g(2.0,3.0) #-> 7.0
# g(2.0, 3) -> "no method matching" error

g(x::Number, y::Number) = 2x - y      # se aplica sempre que x e y são Number
# g(2.0, 3)  -> 1.0

# Note que g(2.0,3) ≢ g(2.0,3.0), já que são chamados métodos diferentes

g(x,y) = "Wrong args, fellow ;)" # -> só é chamado quando x ou y não é Number


methods(g)

# 3 methods for generic function "g":
[1] g(x::Float64, y::Float64) in Main at In[86]:3
[2] g(x::Number, y::Number) in Main at In[86]:7
[3] g(x, y) in Main at In[86]:12

Funções _core_ na linguagem tipicamente têm dezenas - ou mesmo centenas - de métodos:

In [85]:
methods(*)

# 357 methods for generic function "*":
[1] *(x::Bool, z::Complex{Bool}) in Base at complex.jl:290
[2] *(x::Bool, y::Bool) in Base at bool.jl:98
[3] *(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:110
[4] *(x::Bool, z::Complex) in Base at complex.jl:297
[5] *(x::Bool, y::AbstractIrrational) in Base at irrationals.jl:139
[6] *(a::Float16, b::Float16) in Base at float.jl:398
[7] *(x::Float32, y::Float32) in Base at float.jl:404
[8] *(x::Float64, y::Float64) in Base at float.jl:405
[9] *(z::Complex{Bool}, x::Bool) in Base at complex.jl:291
[10] *(z::Complex{Bool}, x::Real) in Base at complex.jl:309
[11] *(r::Regex) in Base at regex.jl:660
[12] *(::Missing, ::Missing) in Base at missing.jl:114
[13] *(::Missing, ::Number) in Base at missing.jl:115
[14] *(d::Missing, x::AbstractString) in Base at missing.jl:174
[15] *(this::Base.Grisu.Float, other::Base.Grisu.Float) in Base.Grisu at grisu/float.jl:140
[16] *(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:414
[17] *(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:515
[18] *(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:514
[19] *(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:513
[20] *(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:473
[21] *(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:532
[22] *(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:534
[23] *(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:564
[24] *(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:557
[25] *(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:551
[26] *(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:410
[27] *(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:379
[28] *(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:386
[29] *(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:394
[30] *(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:402
[31] *(B::BitArray{2}, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /home/ericles/Downloads/julia-1.4.1/share/julia/stdlib/v1.4/LinearAlgebra/src/uniformscaling.jl:195
[32] *(y::AbstractFloat, x::Bool) in Base at bool.jl:112
[33] *(x::T, y::T) where T<:Union{Int128, UInt128} in Base at int.jl:845
[34] *(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:54
[35] *(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:533
[36] *(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:535
[37] *(a::Integer, b::Integer) in Base at int.jl:857
[38] *(x::Real, r::StepRangeLen{#s69,#s68,S} where S where #s68<:Base.TwicePrecision where #s69<:Real) in Base at twiceprecision.jl:479
[39] *(z::Complex, w::Complex) in Base at complex.jl:277
[40] *(z::Complex, x::Bool) in Base at complex.jl:298
[41] *(x::Real, z::Complex{Bool}) in Base at complex.jl:308
[42] *(x::Real, z::Complex) in Base at complex.jl:320
[43] *(z::Complex, x::Real) in Base at complex.jl:321
[44] *(x::Rational, y::Rational) in Base at rational.jl:270
[45] *(a::Integer, index::CartesianIndex{N}) where N in Base.IteratorsMD at multidimensional.jl:120
[46] *(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:390
[47] *(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:398
[48] *(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:406
[49] *(x::AbstractIrrational, y::AbstractIrrational) in Base at irrationals.jl:137
[50] *(y::Real, x::Dates.Period) in Dates at /home/ericles/Downloads/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/periods.jl:91
[51] *(x::Real, A::LinearAlgebra.Hermitian) in LinearAlgebra at /home/ericles/Downloads/julia-1.4.1/share/julia/stdlib/v1.4/LinearAlgebra/src/symmetric.jl:616
[52] *(x::Number) in Base at operators.jl:505
[53] *(x::T, y::T) wher

>_"Multiple dispatch together with the flexible parametric type system give Julia its ability to abstractly express high-level algorithms decoupled from implementation details, yet generate efficient, specialized code to handle each case at run time."_

Não é óbvio porquê _multiple dispatch_ é tão útil, mas a seguir são apresentados alguns padrões idiomáticos, o que deve melhorar o conforto com essa qualidade (para mais, veja [https://docs.julialang.org/en/v1/manual/methods/#Design-Patterns-with-Parametric-Methods-1].

##### Dispatch iterado

In [ ]:
# Primeiro seleciona método necessário para soma elemento a elemento
+(a::Matrix, b::Matrix) = map(+, a, b)

# promote(a,b) "promove" a tupla a uma tupla de tipo único (eg: promote(3.0,4) -> (3.0,4.0))
# o "..." abre a tupla resultante para ser avaliada 
+(a, b) = +(promote(a, b)...)

# Elementos de mesmo tipo podem ser somados diretamente 
+(a::Float64, b::Float64) = Core.add(a, b)

##### Objetos "chamáveis
É possível fazer qualquer objeto Julia "chamável" ao adicionar métodos ao seu tipo (já que métodos estão relacionados com tipos). Esses são às vezes chamados "funtores".

In [96]:
struct Polynomial{R}
    coeffs::Vector{R}
end

# Agora o tipo Polynomial tem um método, e um objeto desse tipo é "chamável"
# veja que a definição é adicionada ao tipo, e não um objeto
function (pol::Polynomial)(x)
    v = pol.coeffs[end]
    for i = (length(pol.coeffs)-1):-1:1
        v = v*x + pol.coeffs[i]
    end
    return v
end

# Define o comportamento para a chamada sem argumentos
(p::Polynomial)() = p(5)

po = Polynomial([1,10,100])

po() # = p(5)

2551

##### Caveats
1. a redefinição ou adição de novo método a uma função pode não ocorrer imediatamente, em runtime (veja [https://docs.julialang.org/en/v1/manual/methods/#Redefining-Methods-1]
2. apenas os argumentos posicionais participam do _dispatching_, mas não os argumentos keyword

#### Exemplos
##### Diferenciação automática
Veja a função a seguir, que computa a raíz quadrada pelo método da babilônia:

In [6]:
function Babilônia(x; n = 5)
    t = (1+x)/2
    for i = 2:n; t=(t + x/t)/2 end
    t
end
Babilônia(pi), sqrt(pi) # veja que os resultados batem

(1.7724538509055159, 1.7724538509055159)

Agora, para a diferenciação automática usaremos o tipo D, de **"número dual"** - conceito estudado pelo algebrista Clifford em 1873. No frigir dos ovos, um número dual vem a ser um par (número, derivada):

In [38]:
# D é uma tupla, x = D((1,2))
struct D <: Number
    f::Tuple{Float64,Float64}
end

Para derivar a função Babilônia, precisamos das regras para soma e para divisão:

In [55]:
import Base: +, /, convert, promote_rule
# (x + y)' = x' + xy'
+(x::D, y::D) = D(x.f .+ y.f)

# (x/y)' = (xy' - yx')/y²
/(x::D, y::D) = D((x.f[1]/y.f[1], (y.f[1]*x.f[2] - x.f[1]*y.f[2])/y.f[1]^2))

# Regra para "converter" um número x do tipo Real para um do tipo D (dual)  
convert(::Type{D}, x::Real) = D((x,zero(x)))

# Declara que, quando 2 variáveis do tipo D e do tipo Number são "promovidas", 
# são promovidas para o tipo D
promote_rule(::Type{D}, ::Type{<:Number}) = D

promote_rule (generic function with 123 methods)

In [62]:
x=π; Babilônia(D((x,1))), 1/(2*√(x))

(D((1.7724538509055159, 0.2820947917738782)), 0.28209479177387814)

Isso não é
1. Diferenciação simbólica;
2. Diferenças finitas;

É apenas Julia fazendo uso de sua estrutura de tipos e multiple dispatch. 

Diferente de diferenciação simbólica, essa forma de diferenciação é no geral muito eficiente em termos de memória e de computação e, diferente de diferenças finitas, a aproximação à derivada da função é boa à nível de máquina.

O tipo Dual é como uma abstração de muito baixo custo. 

Isso é chamado de **forward differentiation**, e pode ser extendida a funções de n variáveis para calcular Jacobiano de forma razoavelmente eficiente, mas a complexidade algorítmica é linear na quantidade de variáveis, então a depender da função, depois de algumas milhares de variáveis o custo pode ser sentido.

Essa técnica, implementada de forma eficiente e para n dimensões, pode ser acessada pela biblioteca [ForwardDiff](https://github.com/JuliaDiff/ForwardDiff.jl).

Uma outra forma de diferenciação automática, por uma técnica diferente chamada **backward differentiation**, que pode ser algoritmicamente mais eficiente, mas tem um overhead maior. Ela se encontra em [BackwardDiff](https://github.com/JuliaDiff/ReverseDiff.jl)

>"ForwardDiff is algorithmically more efficient for differentiating functions where the input dimension is less than the output dimension, while ReverseDiff is algorithmically more efficient for differentiating functions where the output dimension is less than the input dimension.
>Thus, ReverseDiff is generally a better choice for gradients, but Jacobians and Hessians are trickier to determine. For example, optimized methods for computing nested derivatives might use a combination of forward-mode and reverse-mode AD.
>ForwardDiff is often faster than ReverseDiff for lower dimensional gradients (length(input) < 100), or gradients of functions where the number of input parameters is small compared to the number of operations performed on them. ReverseDiff is often faster if your code is expressed as a series of array operations, e.g. a composition of Julia's Base linear algebra methods.
>In general, your choice of algorithms will depend on the function being differentiated, and you should benchmark different methods to see how they fare."


